In [1]:
!nvidia-smi

Wed Jul 26 08:07:53 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.86.01    Driver Version: 515.86.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:02:00.0 Off |                  N/A |
| 34%   74C    P2   112W / 250W |    937MiB / 12288MiB |     98%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  On   | 00000000:04:00.0 Off |                  N/A |
| 22%   

In [2]:
%conda install pytorch torchvision torchaudi -c pytorch

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.

PackagesNotFoundError: The following packages are not available from current channels:

  - torchaudi

Current channels:

  - https://conda.anaconda.org/pytorch/linux-64
  - https://conda.anaconda.org/pytorch/noarch
  - https://repo.anaconda.com/pkgs/main/linux-64
  - https://repo.anaconda.com/pkgs/main/noarch
  - https://repo.anaconda.com/pkgs/r/linux-64
  - https://repo.anaconda.com/pkgs/r/noarch

To search for alternate channels that may provide the conda package you're
looking for, navigate to

    https://anaconda.org

and use the search bar at the top of the page.



Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install torch torchvision torchaudio

Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install torchsummary
%pip install matplotlib
%pip install timm
%pip install einops==0.6.1
%pip install scipy

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [5]:
import torch
torch.cuda.is_available()

/local/musaeed/anaconda3/envs/vit/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [6]:
import torch
import torchvision
print(torch.__version__)
print(torchvision.__version__)

1.8.2+cu102
0.9.2+cu102


In [7]:
import sys
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim

import torchvision.utils
import torchvision.datasets as dsets
from torchsummary import summary

In [8]:
%cd /local/musaeed/.conda/cuda/Medvit_adapter2/

/local/musaeed/.conda/cuda/Medvit_adapter2


In [9]:
!pip install transformers

In [10]:
import MedVit_Adapter
print(dir(MedVit_Adapter))

Could not load the custom kernel for multi-scale deformable attention: CUDA_HOME environment variable is not set. Please set it to your CUDA install root.


['ConvBNReLU', 'ECB', 'F', 'InteractionBlock', 'LTB', 'MedViT_adapter_base', 'MedViT_adapter_large', 'MedViT_adapter_small', 'MedVit_adapter', 'NORM_EPS', 'SpatialPriorModule', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'deform_inputs', 'nn', 'register_model', 'torch']


In [11]:
from MedVit_Adapter import MedViT_adapter_small as small

In [12]:
!pip install medmnist

In [13]:

import medmnist
from medmnist import INFO 
data_flag = 'retinamnist'
# [tissuemnist , pathmnist, chestmnist, dermamnist, octmnisr ,pnemonismnist , retinamnist, bloodmnist, tissuemnist, organcmist, organs ]
download = True

NUM_EPOCHS = 15
BATCH_SIZE = 15
LR = 0.005

info = INFO[data_flag]
task = info['task']
n_channels = info['n_channels']
n_classes = len(info['label'])

DataClass = getattr(medmnist, info['python_class'])

In [14]:

import torchvision.transforms as transforms
#preprocessing
train_transform = transforms.Compose([
    transforms.Resize(224),
    transforms.Lambda(lambda image:image.convert('RGB')),
    #transforms.AugMix(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
]
)
test_transform = transforms.Compose([
    transforms.Resize(224),
    transforms.Lambda(lambda image: image.convert('RGB')),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5],std=[0.5])
])
     

In [15]:
import torch.utils.data as data

# load the data
train_dataset = DataClass(split='train', transform=train_transform, download=download)
val_dataset = DataClass(split='val', transform=train_transform,download=download)
test_dataset = DataClass(split='test', transform=test_transform, download=download)


# encapsulate data into dataloader form
train_loader = data.DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
train_loader_at_eval = data.DataLoader(dataset=val_dataset, batch_size=2*BATCH_SIZE, shuffle=False)
test_loader = data.DataLoader(dataset=test_dataset, batch_size=2*BATCH_SIZE, shuffle=False)

Using downloaded and verified file: /home/CE/musaeed/.medmnist/retinamnist.npz
Using downloaded and verified file: /home/CE/musaeed/.medmnist/retinamnist.npz
Using downloaded and verified file: /home/CE/musaeed/.medmnist/retinamnist.npz


In [16]:

print(train_dataset)
print("+++++++++++++++++")
print(test_dataset)

Dataset RetinaMNIST (retinamnist)
    Number of datapoints: 1080
    Root location: /home/CE/musaeed/.medmnist
    Split: train
    Task: ordinal-regression
    Number of channels: 3
    Meaning of labels: {'0': '0', '1': '1', '2': '2', '3': '3', '4': '4'}
    Number of samples: {'train': 1080, 'val': 120, 'test': 400}
    Description: The RetinaMNIST is based on the DeepDRiD challenge, which provides a dataset of 1,600 retina fundus images. The task is ordinal regression for 5-level grading of diabetic retinopathy severity. We split the source training set with a ratio of 9:1 into training and validation set, and use the source validation set as the test set. The source images of 3×1,736×1,824 are center-cropped and resized into 3×28×28.
    License: CC BY 4.0
+++++++++++++++++
Dataset RetinaMNIST (retinamnist)
    Number of datapoints: 400
    Root location: /home/CE/musaeed/.medmnist
    Split: test
    Task: ordinal-regression
    Number of channels: 3
    Meaning of labels: {'0': 

In [17]:
model = small()

In [18]:
model.proj_head[0] = nn.Linear(in_features=1024, out_features=n_classes, bias=True)

In [19]:
#define loss function and optimizer
if task == 'multi-task, binaty-class':
    criterion = nn.BCEWihtLogLoss()
else:
    criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=LR, momentum=0.9)

In [20]:
criterion

CrossEntropyLoss()

In [27]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [28]:
from torch.nn.functional import softmax
from tqdm import tqdm
# training
for epoch in range(NUM_EPOCHS):

  model = model.to(device)
  model.train()

  for inputs, targets in tqdm(train_loader):
    inputs, targets = inputs.to(device), targets.to(device)
    optimizer.zero_grad()
    outputs = model(inputs.to(torch.float32))

    if task == 'multi-label, binary-class':
      targets = targets.to(torch.float32).unsqueeze(1)
    else:
      targets = targets.to(torch.long)
      targets = targets.view(-1)


    predicted_classes = torch.argmax(outputs, dim=1)
    loss = criterion(outputs, targets)
    loss.backward()
    optimizer.step()

    # Delete tensors to free up memory
    del inputs, targets, outputs, predicted_classes
    # Empty the cache to clear up some more memory
    torch.cuda.empty_cache()

  0%|          | 0/72 [00:00<?, ?it/s]


RuntimeError: Default process group has not been initialized, please make sure to call init_process_group.

In [ ]:

from sklearn.metrics import roc_auc_score, accuracy_score

# switch to evaluation mode
def test(split):
  model.eval()
  # Lists to store actual and predicted values
  actuals = []
  probas = []
  predictions = []

  if split == 'val':
        data_loader = train_loader_at_eval
  else:
        data_loader = test_loader
        
  with torch.no_grad():
      for inputs, targets in tqdm(data_loader):

          inputs, targets = inputs.to(device), targets.to(device)

          if task == 'multi-label, binary-class':
              targets = targets.to(torch.float32).unsqueeze(1)
          else:
              targets = targets.to(torch.long)
              targets = targets.view(-1)

          outputs = model(inputs.to(torch.float32))
          softmax_outputs = softmax(outputs, dim=1)

          _, predicted_classes = torch.max(outputs, 1)

          # Store the actual targets and predicted probabilities
          actuals.extend(targets.cpu().numpy())
          probas.extend(softmax_outputs.detach().cpu().numpy())
          # Probability of positive class
          predictions.extend(predicted_classes.cpu().numpy())
  print('\n')
  auc = roc_auc_score(actuals, probas, multi_class='ovr')
  accuracy = accuracy_score(actuals, predictions)

  print('AUC of the model:', auc)
  print('Accuracy of the model:', accuracy)

print('==> Evaluating...')
test('val')
test('test')